# 2.3 Data Manipulations

__Content:__
 - 2.3.1 Groupby: split-apply-combine
 - 2.3.2 Merging dataframes
 - 2.3.3 Melting dataframes (wide-form to long-form)
 - 2.3.4 Exercises

Import libraries

In [1]:
import numpy as np
import pandas as pd

In this Unit 2.3, we will be using the MovieLens datasets: http://files.grouplens.org/datasets/movielens/ml-100k/README

Citation: F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets:
History and Context. ACM Transactions on Interactive Intelligent
Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.
DOI=http://dx.doi.org/10.1145/2827872

## 2.3.1 Groupby: split-apply-combine

Import data set from url: http://files.grouplens.org/datasets/movielens/ml-100k/u.user

Description: Demographic information about MovieLens users. This is a tab separated list of user_id | age | gender | occupation | zip_code

In [11]:
user = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.user',sep='|',header=None)
user.columns = ['user_id','age','gender','occupation','zip_code']
user.tail()

,user_id,age,gender,occupation,zip_code
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209
942,943,22,M,student,77841


In [10]:
# Any missing values?
user.isnull().sum()

user_id       0
age           0
gender        0
occupation    0
zip_code      0
dtype: int64

In [3]:
# How many unique occupations are there in the data set?
user['occupation'].unique()

array(['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor'], dtype=object)

In [4]:
# The number of unique values.
user['occupation'].nunique()

21

In [13]:
# How many users for each occupation?
user['occupation'].value_counts()

student          196
other            105
educator          95
administrator     79
engineer          67
programmer        66
librarian         51
writer            45
executive         32
scientist         31
artist            28
technician        27
marketing         26
entertainment     18
healthcare        16
retired           14
salesman          12
lawyer            12
none               9
homemaker          7
doctor             7
Name: occupation, dtype: int64

Groupby method allows us to group rows of data together and call aggregate functions.

In [18]:
# How many users for each occupation? Use of groupby method:
user.groupby('occupation').count()

,user_id,age,gender,zip_code
occupation,,,,
administrator,79,79,79,79
artist,28,28,28,28
doctor,7,7,7,7
educator,95,95,95,95
engineer,67,67,67,67
entertainment,18,18,18,18
executive,32,32,32,32
healthcare,16,16,16,16
homemaker,7,7,7,7


In [14]:
# What is the average age for all users?
user['age'].mean()

34.05196182396607

In [17]:
# What is the average age for each occupation? Sort by descending order.
user.groupby('occupation')['age'].mean().sort_values(ascending=False)

occupation
retired          63.071429
doctor           43.571429
educator         42.010526
healthcare       41.562500
librarian        40.000000
administrator    38.746835
executive        38.718750
marketing        37.615385
lawyer           36.750000
engineer         36.388060
writer           36.311111
salesman         35.666667
scientist        35.548387
other            34.523810
technician       33.148148
programmer       33.121212
homemaker        32.571429
artist           31.392857
entertainment    29.222222
none             26.555556
student          22.081633
Name: age, dtype: float64

In [237]:
# Get all summarized statistics about the age for each occupation.
user.groupby('occupation')['age'].describe()

,count,mean,std,min,25%,50%,75%,max
occupation,,,,,,,,
administrator,79.0,38.746835,11.123397,21.0,30.00,37.0,48.00,70.0
artist,28.0,31.392857,8.668116,19.0,25.00,30.0,39.00,48.0
doctor,7.0,43.571429,12.501428,28.0,35.50,45.0,48.50,64.0
educator,95.0,42.010526,10.413264,23.0,33.00,42.0,50.00,63.0
engineer,67.0,36.388060,11.199236,22.0,28.00,36.0,43.00,70.0
entertainment,18.0,29.222222,10.056052,15.0,23.00,25.0,33.00,50.0
executive,32.0,38.718750,10.608075,22.0,31.00,38.5,46.00,69.0
healthcare,16.0,41.562500,11.313524,22.0,30.50,44.5,49.25,62.0
homemaker,7.0,32.571429,10.737119,20.0,24.50,32.0,38.50,50.0


In [244]:
# Apply customised function 
# Example: Find the range of age for each occupation
user.groupby('occupation')['age'].apply(lambda x: x.max() - x.min())

occupation
administrator    49
artist           29
doctor           36
educator         40
engineer         48
entertainment    35
executive        47
healthcare       40
homemaker        30
lawyer           32
librarian        46
marketing        31
none             44
other            51
programmer       43
retired          22
salesman         48
scientist        32
student          35
technician       34
writer           42
Name: age, dtype: int64

In [25]:
# Calculate the mean age for each combination of occupation and gender.
user.groupby(['occupation','gender'])['age'].mean()
# Output is a multi-index series

occupation     gender
administrator  F         40.638889
               M         37.162791
artist         F         30.307692
               M         32.333333
doctor         M         43.571429
educator       F         39.115385
               M         43.101449
engineer       F         29.500000
               M         36.600000
entertainment  F         31.000000
               M         29.000000
executive      F         44.000000
               M         38.172414
healthcare     F         39.818182
               M         45.400000
homemaker      F         34.166667
               M         23.000000
lawyer         F         39.500000
               M         36.200000
librarian      F         40.000000
               M         40.000000
marketing      F         37.200000
               M         37.875000
none           F         36.500000
               M         18.600000
other          F         35.472222
               M         34.028986
programmer     F         32.16666

In [23]:
# Selecting data in a multi-index series
user1 = user.groupby(['occupation','gender'])['age'].mean()
user1['student']['M']

22.669117647058822

In [26]:
# Unstacking the multi-index series
user.groupby(['occupation','gender'])['age'].mean().unstack()

gender,F,M
occupation,,
administrator,40.638889,37.162791
artist,30.307692,32.333333
doctor,NaN,43.571429
educator,39.115385,43.101449
engineer,29.500000,36.600000
entertainment,31.000000,29.000000
executive,44.000000,38.172414
healthcare,39.818182,45.400000
homemaker,34.166667,23.000000


In [25]:
# Unstacking multi-index series will give a data frame
user2 = user.groupby(['occupation','gender'])['age'].mean().unstack()
user2.loc['student']['M']

22.669117647058822

Analysing categorical variables.

In [22]:
# How many male and female users? 
user.groupby('gender')['age'].count()

gender
F    273
M    670
Name: age, dtype: int64

In [24]:
# What are the percentages of male and female users?
100*user.groupby('gender')['age'].count()/len(user)

gender
F    28.950159
M    71.049841
Name: age, dtype: float64

In [36]:
# What are the percentages of male users for each occupation?
# First, create a new column to convert string to numbers/boolean
user['gen'] = user['gender'].apply(lambda x: 1 if x=='M' else 0)
user.head()

,age,gender,occupation,zip_code,gen
user_id,,,,,
1,24,M,technician,85711,1
2,53,F,other,94043,0
3,23,M,writer,32067,1
4,24,M,technician,43537,1
5,33,F,other,15213,0


In [48]:
user.groupby(['occupation'])['gen'].mean()*100

occupation
administrator     54.430380
artist            53.571429
doctor           100.000000
educator          72.631579
engineer          97.014925
entertainment     88.888889
executive         90.625000
healthcare        31.250000
homemaker         14.285714
lawyer            83.333333
librarian         43.137255
marketing         61.538462
none              55.555556
other             65.714286
programmer        90.909091
retired           92.857143
salesman          75.000000
scientist         90.322581
student           69.387755
technician        96.296296
writer            57.777778
Name: gen, dtype: float64

In [45]:
# Find the percentages of male and female users for each occupation.
users = 100*user.groupby(['occupation','gender'])['gender'].count()/user.groupby(['occupation'])['gender'].count()
users.unstack()

gender,F,M
occupation,,
administrator,45.569620,54.430380
artist,46.428571,53.571429
doctor,NaN,100.000000
educator,27.368421,72.631579
engineer,2.985075,97.014925
entertainment,11.111111,88.888889
executive,9.375000,90.625000
healthcare,68.750000,31.250000
homemaker,85.714286,14.285714


Suppose that the zip_code can be classified as follows:

First digit of zip_code => Region
- 0, 1 => 'North';
- 2, 3 => 'South'; 
- 4, 5 => 'East';
- 6, 7 => 'West';
- 8, 9 => 'Central'

In [4]:
# Write a function to return a new column with 'Region'
#def get_region(x):
#    x = pd.to_numeric(x, errors='coerce')
#    if x in [0,1]:
#        return 'North'
#    elif x in [2,3]:
#        return 'South'
#    elif x in [4,5]:
#        return 'East'
#    elif x in [6,7]:
#        return 'West'
#    elif x in [8,9]:
#        return 'Central'
#    else:
#        return None

#user['zip'] = user['zip_code'].str[0].astype('int')
#user['Region'] = user['zip'].apply(get_region)
#user.head()

In [ ]:
#user['Region'] = user['zip_code'].str[0].apply(get_region)

In [3]:
#user.groupby(['Region','gender'])['age'].count().unstack()

In [2]:
#def get_age_group(x):
#    if x < 25:
#        return 'Young Adults'
#    elif x <= 60:
#        return 'Adults'
#    else:
#        return 'Senior Citizens'

In [179]:
#user['age_group'] = user['age'].apply(get_age_group)
#user.head()

,age,gender,occupation,zip_code,gen,Region,zip,age_group
user_id,,,,,,,,
1,24,M,technician,85711,1,Central,8,Young Adults
2,53,F,other,94043,0,Central,9,Adults
3,23,M,writer,32067,1,South,3,Young Adults
4,24,M,technician,43537,1,East,4,Young Adults
5,33,F,other,15213,0,North,1,Adults


In [1]:
#user.groupby(['age_group','gender'])['age'].describe().columns

## 2.3.2 Merging dataframes

Import data from url: http://files.grouplens.org/datasets/movielens/ml-100k/u.data

This is the data set of 100,000 ratings by 943 users on 1682 movies. Each user has rated at least 20 movies. 

The data is randomly ordered. This is a tab separated list of user_id | movie_id | rating | timestamp.

In [269]:
data = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.data',sep='\t',header=None)
data.columns = ['user_id','movie_id','rating','timestamp']
data.tail()

,user_id,movie_id,rating,timestamp
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156
99999,12,203,3,879959583


Import data from url: http://files.grouplens.org/datasets/movielens/ml-100k/u.item

This dataset contains information about the movies. There are 24 columns but we will be using the first 3 columns only: movie_id | movie_title | release date

In [271]:
movie = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.item',sep='|',header=None,encoding='latin-1')
movie = movie.loc[:,0:2]
movie.columns = ['movie_id','movie_title','release_date']
movie.tail()

,movie_id,movie_title,release_date
1677,1678,Mat' i syn (1997),06-Feb-1998
1678,1679,B. Monkey (1998),06-Feb-1998
1679,1680,Sliding Doors (1998),01-Jan-1998
1680,1681,You So Crazy (1994),01-Jan-1994
1681,1682,Scream of Stone (Schrei aus Stein) (1991),08-Mar-1996


Merge two dataframes based on 'movie_id'.

In [306]:
df = pd.merge(data,movie,on='movie_id')
df.tail()

,user_id,movie_id,rating,timestamp,movie_title,release_date
99995,840,1674,4,891211682,Mamma Roma (1962),01-Jan-1962
99996,655,1640,3,888474646,"Eighth Day, The (1996)",01-Nov-1996
99997,655,1637,3,888984255,Girls Town (1996),23-Aug-1996
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1...",02-Feb-1996
99999,655,1641,3,887427810,Dadetown (1995),18-Sep-1996


Find the average and the number of ratings for all movies.

In [319]:
df.groupby('movie_title')['rating'].agg([np.mean,np.size]).sort_values(by='mean',ascending=False).head()

,mean,size
movie_title,,
They Made Me a Criminal (1939),5.0,1
Marlene Dietrich: Shadow and Light (1996),5.0,1
"Saint of Fort Washington, The (1993)",5.0,2
Someone Else's America (1995),5.0,1
Star Kid (1997),5.0,3


__Challenge!__ Out of the top-100 most rated movies, which movie has the highest average rating?

__Challenge!!__ Can you get the answer in one line of code?

In [335]:
df1 = df.groupby('movie_title')['rating'].agg([np.mean,np.size]).sort_values(by='size',ascending=False).head(100)
df1.sort_values(by='mean',ascending=False).head()

,mean,size
movie_title,,
Schindler's List (1993),4.466443,298
Casablanca (1942),4.456790,243
"Shawshank Redemption, The (1994)",4.445230,283
"Usual Suspects, The (1995)",4.385768,267
Star Wars (1977),4.358491,583


In [331]:
df.groupby('movie_title')['rating'].agg([np.mean,np.size]).sort_values(by='size',ascending=False).head(100).idxmax()[0]

"Schindler's List (1993)"

## 2.3.3 Wide-form vs Long-form Dataframe

In [3]:
quiz = pd.read_excel('Quiz.xlsx')
quiz.head()
# This data frame is in wide format

,ID,Quiz1,Quiz2,Quiz3,Quiz4,Quiz5
0,p1,1,10,5,10,10
1,p2,2,9,6,9,10
2,p3,3,8,7,8,10
3,p4,4,7,8,7,10
4,p5,5,6,9,6,10


In [7]:
# Melt the wide-form df to long-form
quizlong = pd.melt(quiz,id_vars=['ID'],value_vars=['Quiz1','Quiz2','Quiz3','Quiz4','Quiz5'],var_name='Quiz',value_name='Score')
quizlong.head()

,ID,Quiz,Score
0,p1,Quiz1,1
1,p2,Quiz1,2
2,p3,Quiz1,3
3,p4,Quiz1,4
4,p5,Quiz1,5


In [8]:
# Shape of the long-form df
quizlong.shape

(100, 3)

In [38]:
# Change long-form to wide-form by using pivot
quizwide = quizlong.pivot(index='ID',columns='Quiz',values='Score')
quizwide.reset_index().head()

Quiz,ID,Quiz1,Quiz2,Quiz3,Quiz4,Quiz5
0,p1,1,10,5,10,10
1,p10,10,1,10,8,10
2,p11,10,1,10,8,9
3,p12,9,2,9,8,9
4,p13,8,3,8,8,9


In [31]:
# movie id | movie title | release date | video release date |
#              IMDb URL | unknown | Action | Adventure | Animation |
#              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
#              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
#              Thriller | War | Western |

In [30]:
#mv = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.item',sep='|',header=None,encoding='latin-1')
#mv.tail()

In [29]:
#mv.columns= ['movie id' , 'movie title' , 'release date', 'video release date', 'IMDB URL' , 'Unknown', 'Action',
#              'Adventure' , 'Animation', "Children" , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
#              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' , 'Thriller' , 'War' , 'Western']
#mv.head()

In [28]:
#mvlong = pd.melt(mv, id_vars=['movie id','movie title','release date','video release date','IMDB URL'], 
#                   value_vars=['Unknown', 'Action', 'Adventure' , 'Animation', "Children" , 'Comedy' , 'Crime' , 
#                               'Documentary' , 'Drama' , 'Fantasy' , 'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 
#                               'Romance' , 'Sci-Fi' , 'Thriller' , 'War' , 'Western'],
#                   var_name='Genre', value_name='Value')
#mvlong.head(10)

In [26]:
#mvlong.shape

In [25]:
#mv[mv['movie title'] == 'Star Wars (1977)']

In [24]:
#mvlong[mvlong['movie title'] == 'Star Wars (1977)']

In [23]:
# Which is the most popular movie genre?
# mvlong.groupby('Genre')['Value'].sum()

## 2.3.4 Exercises

In [19]:
df = pd.read_excel('Marks.xlsx')
df.head()

,Student_Id,Subject_Class,Student_Class,ICA,MST,Exam
0,p1,Class01,DARE/FT/1A/01,81,65,74
1,p2,Class01,DARE/FT/1A/01,75,79,94
2,p3,Class01,DARE/FT/1A/01,85,92,73
3,p4,Class01,DARE/FT/1A/01,87,66,73
4,p5,Class01,DARE/FT/1A/01,82,52,90


In [20]:
# Insert a new column Total = 0.25*ICA + 0.25*MST + 0.5* Exam
df['Total'] = 0.25*df['ICA']+0.25*df['MST']+0.5*df['Exam']
df.sample(5)

,Student_Id,Subject_Class,Student_Class,ICA,MST,Exam,Total
60,p61,Class04,DMRO/FT/2A/01,88,61,68,71.25
36,p37,Class03,DCHE/FT/2A/05,87,86,87,86.75
30,p31,Class02,DCPE/FT/2B/06,71,75,87,80.00
43,p44,Class03,DAPC/FT/2A/02,48,76,74,68.00
22,p23,Class02,DEEE/FT/2A/05,84,65,49,61.75


In [21]:
df['Course'] = df['Student_Class'].str.split('/').str[0]
df.sample(5)

,Student_Id,Subject_Class,Student_Class,ICA,MST,Exam,Total,Course
45,p46,Class03,DAPC/FT/2A/04,65,66,48,56.75,DAPC
8,p9,Class01,DARE/FT/1A/01,57,62,76,67.75,DARE
19,p20,Class02,DEEE/FT/2A/02,76,59,83,75.25,DEEE
33,p34,Class03,DCHE/FT/2A/02,82,92,54,70.50,DCHE
75,p76,Class05,DASE/FT/1A/02,92,66,58,68.50,DASE


In [22]:
df['Year'] = df['Student_Class'].str.split('/').str[2].str[0]
df.sample(5)

,Student_Id,Subject_Class,Student_Class,ICA,MST,Exam,Total,Course,Year
27,p28,Class02,DCPE/FT/2B/03,80,89,78,81.25,DCPE,2
50,p51,Class03,DAPC/FT/2A/04,94,60,72,74.50,DAPC,2
4,p5,Class01,DARE/FT/1A/01,82,52,90,78.50,DARE,1
51,p52,Class03,DAPC/FT/2A/05,78,73,65,70.25,DAPC,2
55,p56,Class04,DME/FT/2B/22,90,71,70,75.25,DME,2


In [23]:
school = pd.DataFrame({'Course':['DARE','DASE','DCHE','DAPC','DCPE','DEEE','DME','DMRO','DBEN'],
                      'School':['MAE','EEE','CLS','CLS','EEE','EEE','MAE','MAE','MAE']})
school

,Course,School
0,DARE,MAE
1,DASE,EEE
2,DCHE,CLS
3,DAPC,CLS
4,DCPE,EEE
5,DEEE,EEE
6,DME,MAE
7,DMRO,MAE
8,DBEN,MAE


In [24]:
# Merge the two data frames to match Course to School
df1 = pd.merge(df,school,on='Course')
df1.sample(5)

,Student_Id,Subject_Class,Student_Class,ICA,MST,Exam,Total,Course,Year,School
7,p8,Class01,DARE/FT/1A/01,84,91,95,91.25,DARE,1,MAE
43,p44,Class03,DAPC/FT/2A/02,48,76,74,68.00,DAPC,2,CLS
84,p85,Class05,DASE/FT/2A/03,70,71,58,64.25,DASE,2,EEE
42,p43,Class03,DAPC/FT/2A/01,90,78,89,86.50,DAPC,2,CLS
83,p84,Class05,DASE/FT/2A/02,79,80,90,84.75,DASE,2,EEE


In [25]:
# Basic descriptive stats
df1.describe()

,ICA,MST,Exam,Total
count,88.000000,88.000000,88.000000,88.000000
mean,77.079545,70.397727,74.920455,74.329545
std,11.949339,13.253097,14.089388,8.007140
min,48.000000,31.000000,41.000000,55.000000
25%,69.750000,62.750000,64.000000,68.437500
50%,78.500000,71.000000,76.500000,74.375000
75%,86.250000,79.000000,87.000000,80.562500
max,98.000000,95.000000,96.000000,91.250000


In [27]:
# Basic descriptive stats groupby School & Course
df1.groupby(['School','Course'])['Total'].describe()

count       mean       std    min      25%     50%      75%  \
School Course                                                                
CLS    DAPC     10.0  74.275000  9.132854  56.75  70.2500  73.750  80.1250   
       DCHE     10.0  75.800000  8.785879  63.50  68.8125  76.375  83.2500   
EEE    DASE     18.0  74.041667  9.863871  55.00  68.3125  72.500  82.0000   
       DCPE      7.0  78.392857  3.920717  70.00  78.2500  80.000  80.5000   
       DEEE      7.0  67.892857  5.537481  61.75  63.6250  66.250  72.3750   
MAE    DARE     18.0  75.388889  7.042530  64.75  70.7500  74.375  80.1875   
       DBEN      6.0  75.875000  7.427567  64.50  71.2500  78.000  81.1875   
       DME       6.0  71.333333  7.976006  61.00  66.3125  71.375  75.1250   
       DMRO      6.0  73.875000  7.026290  63.00  71.6875  73.625  76.6875   

                 max  
School Course         
CLS    DAPC    87.00  
       DCHE    87.50  
EEE    DASE    88.50  
       DCPE    81.25  
       DEEE    75.25  
MAE    DARE    91.25  
       DBEN    83.50  
       DME     83.25  
       DMRO    84.25

In [28]:
# Customising aggregated values
df1.groupby(['School','Course'])['Exam'].agg([np.size,np.mean,np.min,np.max,np.std])

size       mean  amin  amax        std
School Course                                        
CLS    DAPC      10  74.000000    48    94  13.399834
       DCHE      10  73.000000    51    89  14.839886
EEE    DASE      18  73.166667    41    95  17.507141
       DCPE       7  80.714286    68    95  10.355583
       DEEE       7  69.285714    49    83  15.315725
MAE    DARE      18  79.500000    62    95  10.584395
       DBEN       6  81.166667    62    96  12.006942
       DME        6  68.500000    51    85  13.575714
       DMRO       6  71.166667    47    94  16.509593

In [30]:
df1.groupby(['Subject_Class'])['Total'].agg({'N':np.size,'Mean':np.mean,'Min':np.min,'Max':np.max,'SD':np.std,
                                      'Pass_Rate':lambda x: 100*np.mean(x>=50),
                                     'B+&Above': lambda x: 100*sum(x>=75)/len(x)})

C:\Windows\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,N,Mean,Min,Max,SD,Pass_Rate,B+&Above
Subject_Class,,,,,,,
Class01,18.0,75.388889,64.75,91.25,7.042530,100.0,38.888889
Class02,14.0,73.142857,61.75,81.25,7.136522,100.0,50.000000
Class03,20.0,75.037500,56.75,87.50,8.757055,100.0,45.000000
Class04,18.0,73.694444,61.00,84.25,7.287961,100.0,44.444444
Class05,18.0,74.041667,55.00,88.50,9.863871,100.0,44.444444


In [31]:
df1.groupby(['School','Course'])['Total'].agg({'N':np.size,'Mean':np.mean,'Min':np.min,'Max':np.max,'SD':np.std,
                                      'Pass_Rate':lambda x: 100*np.mean(x>=50),
                                     'B+&Above': lambda x: 100*sum(x>=75)/len(x)})

C:\Windows\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


N       Mean    Min    Max        SD  Pass_Rate   B+&Above
School Course                                                               
CLS    DAPC    10.0  74.275000  56.75  87.00  9.132854      100.0  30.000000
       DCHE    10.0  75.800000  63.50  87.50  8.785879      100.0  60.000000
EEE    DASE    18.0  74.041667  55.00  88.50  9.863871      100.0  44.444444
       DCPE     7.0  78.392857  70.00  81.25  3.920717      100.0  85.714286
       DEEE     7.0  67.892857  61.75  75.25  5.537481      100.0  14.285714
MAE    DARE    18.0  75.388889  64.75  91.25  7.042530      100.0  38.888889
       DBEN     6.0  75.875000  64.50  83.50  7.427567      100.0  66.666667
       DME      6.0  71.333333  61.00  83.25  7.976006      100.0  33.333333
       DMRO     6.0  73.875000  63.00  84.25  7.026290      100.0  33.333333

In [32]:
df1.groupby(['Year'])['Total'].agg({'N':np.size,'Mean':np.mean,'Min':np.min,'Max':np.max,'SD':np.std,
                                      'Pass_Rate':lambda x: 100*np.mean(x>=50),
                                     'B+&Above': lambda x: 100*sum(x>=75)/len(x)})

C:\Windows\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,N,Mean,Min,Max,SD,Pass_Rate,B+&Above
Year,,,,,,,
1,30.0,75.20000,55.00,91.25,8.605181,100.0,43.333333
2,58.0,73.87931,56.75,87.50,7.718774,100.0,44.827586


In [37]:
# Compute correlations among the variables
df1.corr()

,ICA,MST,Exam,Total
ICA,1.000000,-0.081565,0.125181,0.449468
MST,-0.081565,1.000000,-0.194284,0.212428
Exam,0.125181,-0.194284,1.000000,0.846112
Total,0.449468,0.212428,0.846112,1.000000


In [38]:
# Export cleaned data frame to Excel
df1.to_excel('Marks_Clean.xlsx')

In [33]:
#def pass_rate(x):
#    return 100*np.mean(x>=50)

In [34]:
#x=pd.Series([1,50,100])
#pass_rate(x)

In [35]:
#def BPlus(x):
#    return 100*sum(x>=75)/len(x)

In [36]:
#df1.groupby(['Year'])['Marks'].agg([pass_rate,BPlus]).rename(columns={'pass_rate':'Pass Rate','BPlus':'B+&Above'})